In [1]:
from keras.layers import Input, Dense, Flatten, Lambda, Activation, Dropout
from keras.layers import MaxPooling2D, Conv2D, BatchNormalization
from keras.models import Model
import keras.backend as K
import keras

import tensorflow as tf

from sklearn.metrics import explained_variance_score

import pandas as pd

import cv2 as cv

import numpy as np

Using TensorFlow backend.


# Model Definition

## Encoder

### Doesn't work
The role of this model is to create feature vector from raw images.

Borrow from https://github.com/uzh-rpg/rpg_public_dronet

Note:
    Layer naming convention: `layer_name(str) + stage(int) + block(str)`

___

### [MAY14] NVIDIA style
https://github.com/upul/Behavioral-Cloning/blob/master/model.py

In [3]:
def encoder(X_input):
    """
    Define encoder
    """
    X = Lambda(lambda x: x / 127.5 - 1.0)(X_input)
    
    # First convolution block
    X = Conv2D(24, (5, 5), strides=(2, 2), padding="same")(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(2, 2), strides=(1, 1))(X)
    
    # Second convolution block
    X = Conv2D(36, (5, 5), strides=(2, 2), padding="same")(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(2, 2), strides=(1, 1))(X)
    
    # Third convolution block
    X = Conv2D(48, (5, 5), strides=(2, 2), padding="same")(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(2, 2), strides=(1, 1))(X)
    
    # Fourth convolution block
    X = Conv2D(64, (3, 3), strides=(1, 1), padding="same")(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(2, 2), strides=(1, 1))(X)
    
    # Fifth convolution block
    X = Conv2D(64, (3, 3), strides=(1, 1), padding="same")(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(2, 2), strides=(1, 1))(X)
    
    y = Flatten()(X)
    
    return y


def hybrid_model(img_shape):
    """
    Define encoder-decoder model
    """
    # Input layer
    X_input_0 = Input(shape=img_shape)
    X_input_1 = Input(shape=img_shape)
    X_input_2 = Input(shape=img_shape)
    
    # encode
    en_y_0 = encoder(X_input_0)
    en_y_1 = encoder(X_input_1)
    en_y_2 = encoder(X_input_2)
    
    en_y = keras.layers.concatenate([en_y_0, en_y_1, en_y_2])
    
    # decode
    X = Dropout(0.5)(en_y)
    
    X = Dense(1164, activation='relu')(X)
    
    X = Dropout(0.5)(X)
    
    X = Dense(500, activation='relu')(X)
    
    X = Dense(200, activation='relu')(X)
        
    y = Dense(25, activation=None)(X)
    
    # define model
    model = Model(inputs=[X_input_0, X_input_1, X_input_2], outputs=[y])
    print(model.summary())
    
    return model
    

### Create model & load weights

In [3]:
IMAGE_SHAPE = (64, 64, 1)
model = hybrid_model(IMAGE_SHAPE)
# model.load_weights('./model/full_hybrid_MAY14_MAE_ext.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 64, 64, 1)    0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)    

___

## [MAY15] Seed paper style

Input is a RGB image size 52x265x3

In [2]:
def encoder_seed_paper(X_input):
    """
    Define the CNN for extracting features from each image
    """
    # Normalize layer
    X = Lambda(lambda x: x / 127.5 - 1.0)(X_input)
    
    # First block
    X = Conv2D(24, (3, 3), strides=(1, 1), padding='valid', 
               activation='relu', kernel_initializer='he_normal')(X)
    X = Conv2D(24, (3, 3), strides=(1, 1), padding='valid', 
               activation='relu', kernel_initializer='he_normal')(X)
    X = BatchNormalization()(X) 
    X = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(X)
    
    # Second block
    X = Conv2D(36, (3, 3), strides=(1, 1), padding='valid', 
               activation='relu', kernel_initializer='he_normal')(X)
    X = Conv2D(36, (3, 3), strides=(1, 1), padding='valid', 
               activation='relu', kernel_initializer='he_normal')(X)
    X = BatchNormalization()(X)
    X = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(X)
    
    # Third block
    X = Conv2D(48, (3, 3), strides=(1, 1), padding='valid', 
               activation='relu', kernel_initializer='he_normal')(X)
    X = Conv2D(48, (3, 3), strides=(1, 1), padding='valid', 
               activation='relu', kernel_initializer='he_normal')(X)
    X = BatchNormalization()(X)
    X = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid')(X)
    
    # Fourth block
    X = Conv2D(64, (3, 3), strides=(1, 1), padding='same', 
               activation='relu', kernel_initializer='he_normal')(X)
    X = Conv2D(64, (3, 3), strides=(1, 1), padding='valid', 
               activation='relu', kernel_initializer='he_normal')(X)
    
    X = Flatten()(X)
    
    return X


def hybrid_model_seed_paper(image_shape):
    """
    Define the encoder-decoder model, seed paper style
    """
    # Input layer
    X_input_0 = Input(shape=image_shape)
    X_input_1 = Input(shape=image_shape)
    X_input_2 = Input(shape=image_shape)
    
    # Encode
    en_y_0 = encoder_seed_paper(X_input_0)
    en_y_1 = encoder_seed_paper(X_input_1)
    en_y_2 = encoder_seed_paper(X_input_2)
    
    # stack 3 feautres vectors 
    en_y = keras.layers.concatenate([en_y_0, en_y_1, en_y_2])
    
    # Decode
    X = Dropout(0.5)(en_y)
    
    X = Dense(1164, activation='relu')(en_y)
    
    X = Dense(500, activation='relu')(X)
    
    X = Dense(200, activation='relu')(X)
        
    y = Dense(25, activation=None)(X)
    
    # define model
    model = Model(inputs=[X_input_0, X_input_1, X_input_2], outputs=[y])
    print(model.summary())
    
    return model


def hybrid_model_LSTM(image_shape):
    """
    Define the encoder-decoder model, seed paper style, but with an LSTM put on top
    instead of 4 Dense layers
    """
    # Input layer
    X_input_0 = Input(shape=image_shape)
    X_input_1 = Input(shape=image_shape)
    X_input_2 = Input(shape=image_shape)
    
    # Encode
    en_y_0 = encoder_seed_paper(X_input_0)
    en_y_1 = encoder_seed_paper(X_input_1)
    en_y_2 = encoder_seed_paper(X_input_2)
    
    # stack 3 feautres vectors 
    en_y_0 = keras.layers.Reshape((1, -1))(en_y_0)
    en_y_1 = keras.layers.Reshape((1, -1))(en_y_1)
    en_y_2 = keras.layers.Reshape((1, -1))(en_y_2)
    
    en_y = keras.layers.concatenate([en_y_0, en_y_1, en_y_2], axis=1)
    
    # Decode
    X = keras.layers.LSTM(128, return_sequences=False)(en_y)
    y = Dense(25, activation=None)(X)
    
    # define model
    model = Model(inputs=[X_input_0, X_input_1, X_input_2], outputs=[y])
    print(model.summary())
    
    return model
    

### Create model & load weights

In [3]:
IMAGE_SHAPE = (52, 265, 1)
model = hybrid_model_LSTM(IMAGE_SHAPE)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 52, 265, 1)   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 52, 265, 1)   0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 52, 265, 1)   0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda) 

In [4]:
# load weights
# model.load_weights("./model/seed_BatchNorm_1Channel.h5")

In [4]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true)))
    
    
# def explained_variance(y_true, y_pred):
#     y_true_var = K.var(y_true, axes=-1)
#     subtract_var = K.var(keras.layers.subtract([y_true, y_pred]), axes=-1)
#     return Lambda(lambda x: x[0]/x[1])([tensor1,tensor2])
    
# Compile
model.compile(loss=root_mean_squared_error, optimizer='adam')

# Data Generator

This class preprocesses training examples, form traning batches and fit batches to network 

In [5]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, dataset_csv, img_shape, batch_size=1, shuffle=True):
        """
        Input:
            dataset_csv (string): path to csv contains dataset
            img_shape (tuple): (img_height, img_width, img_channel)
            batch_size (int): size of a training batch
            shuffle (bool): shuffle dataset after 1 epoch
        """
        
        self.df = pd.read_csv(dataset_csv)
        
        # check channel of image
        if img_shape[-1] == 1:
            self.color_img = False
        else:
            self.color_img = True
        
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.img_shape = img_shape
        # invoke on_epoch_end to create shuffle training dataset
        self.on_epoch_end()
    
    def __len__(self):
        """
        Output:
            the number of batches per epoch
        """
        return int(np.floor(len(self.df) / self.batch_size))
    
    def __data_generation(self, list_indexes):
        """
        Input:
            list_indexes (list): list of indexes of training sample in this batch
        
        Output:
            X (list): list of np.ndarray. Each have shape of (batch_size, img_height, img_width, img_channel) 
            y (np.ndarray): label vector, shape (batch_size, 25)
        """
        
        X = [np.zeros((self.batch_size,) + self.img_shape) for i in range(3)]
        
        y = np.zeros((self.batch_size, 25))
        
        img_path_prefix = '/home/user/Bureau/Dataset/udacity/'
        
        # Iterate through each idx in training batch
        for i, idx in enumerate(list_indexes): 
            file_names_list = self.df.iloc[idx].X[2: -2].split("', '")
            # read img & put it in  X_0, X_1, X_2
            for j, name in enumerate(file_names_list):
                # read image
                if not self.color_img:
                    img = cv.imread(img_path_prefix + name, 0)
                else:
                    img = cv.imread(img_path_prefix + name, 1)
                
                # resize & reshape image
                img = np.float32(cv.resize(img, (self.img_shape[1], self.img_shape[0]), 
                                           interpolation=cv.INTER_AREA))
                if len(img.shape) == 2:
                    img = img.reshape((self.img_shape))
                
                # add img to input tensor
                X[j][i, :, :, :] = img
            
            # get label
            y[i, :] = np.array([float(angle) for angle in self.df.iloc[idx].steering_angles[1: -1].split(", ")])
        return X, y
    
    def __getitem__(self, index):
        """
        Generate one batch of data
        
        Input:
            index (int): index of the first training sample
        """
        # Generate indexes of the batch
        list_indexes = self.indexes[index * self.batch_size : (index + 1) * self.batch_size]
        # if (index + 1) * self.batch_size > len(self.indexes), 
        # list_indexes = [index * self.batch_size: len(self.indexes)]

        # Generate data
        X, y = self.__data_generation(list_indexes)

        return X, y
    
    def on_epoch_end(self):
        """
        Updates indexes after each epoch
        """
        self.indexes = np.arange(len(self.df))  # array of indexes of training dataset
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
        
                    

___

# Training

Baseline performance:
* RMSE: 0.104
* EVA: 0.737

In [8]:
# parameters of datagenerator
params = {'img_shape': IMAGE_SHAPE, 
          'batch_size': 3, 
          'shuffle': True}

training_generator = DataGenerator('./data/ext_hybrid_training.csv', **params)
validation_generator = DataGenerator('./data/ext_hybrid_validation.csv', **params)

log_dir="./rev-logs/seed-RNN/"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    epochs=5,
                    callbacks=[tensorboard_callback],
                    verbose=1)

Epoch 1/5
4828/4828 [==============================] - 373s 77ms/step - loss: 0.0980 - val_loss: 0.0900
Epoch 2/5
4828/4828 [==============================] - 373s 77ms/step - loss: 0.0977 - val_loss: 0.0909
Epoch 3/5
4828/4828 [==============================] - 366s 76ms/step - loss: 0.0983 - val_loss: 0.0906
Epoch 4/5
4828/4828 [==============================] - 375s 78ms/step - loss: 0.0981 - val_loss: 0.0910
Epoch 5/5
4828/4828 [==============================] - 377s 78ms/step - loss: 0.0982 - val_loss: 0.0911


# Save model

In [9]:
# save encoder structure & weights
def save_model(model, name):
    # serialize model to JSON
    #  the keras model which is trained is defined as 'model' in this example
    model_json = model.to_json()


    with open("./model/%s.json" % name, "w") as json_file:
        json_file.write(model_json)

    # serialize weights to HDF5
    model.save_weights("./model/%s.h5" % name)
    
    
# save decoder weights
save_model(model, 'seed_RNN_cont')

In [10]:
del model